<a href="https://colab.research.google.com/github/jacquesbilombe/Plustrategy/blob/main/datafetcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For more information on the API, check this [link](https://pypi.org/project/sportradar/).

In [7]:
import os
import sys
import csv
import json
import datetime
import requests
import pandas as pd

from bs4 import BeautifulSoup

In [2]:
cookies = {
    '_gcl_au': '1.1.1494853104.1715364312',
    '_ga': 'GA1.1.1918003453.1715364313',
    '__qca': 'P0-1691788869-1715364322655',
    'FCNEC': '%5B%5B%22AKsRol9XuTvtcA-KtSeor_fgzdQW58RK_BpfZB7Sdj5I2XedsLxKLTo20tcRe8s8L53_TyoQtuRWeKKXFto4oIVAygDrfxFElND8Xx-JTSyxHbl3W_lqOhjJ7snWZ73Xr-3PupQZsWVMa3yJ19Vv92Tc6i2muqe-rg%3D%3D%22%5D%5D',
    '__gads': 'ID=b5947f16edb0403d:T=1715364320:RT=1715366201:S=ALNI_MaJIyMoeqZaO--UX-FZHBggRDNdJA',
    '__gpi': 'UID=00000a257e3bc349:T=1715364320:RT=1715366201:S=ALNI_MbCel5XYfhe-MDCNwKoU5oysKgFOA',
    '__eoi': 'ID=934a22df78350c96:T=1715364320:RT=1715366201:S=AA-AfjZuN6AaSh_XAQiV6aYPqXtz',
    '_ga_3KF4XTPHC4': 'GS1.1.1715364313.1.1.1715366296.60.0.0',
    '_ga_QH2YGS7BB4': 'GS1.1.1715364313.1.1.1715366296.0.0.0',
    '_ga_HNQ9P9MGZR': 'GS1.1.1715364319.1.1.1715366296.57.0.0',
}

headers = {
    'accept': '*/*',
    'accept-language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7,pt-BR;q=0.6,pt;q=0.5,fr-CD;q=0.4,ja-JP;q=0.3,ja;q=0.2',
    'cache-control': 'max-age=0',
    # 'cookie': '_gcl_au=1.1.1494853104.1715364312; _ga=GA1.1.1918003453.1715364313; __qca=P0-1691788869-1715364322655; FCNEC=%5B%5B%22AKsRol9XuTvtcA-KtSeor_fgzdQW58RK_BpfZB7Sdj5I2XedsLxKLTo20tcRe8s8L53_TyoQtuRWeKKXFto4oIVAygDrfxFElND8Xx-JTSyxHbl3W_lqOhjJ7snWZ73Xr-3PupQZsWVMa3yJ19Vv92Tc6i2muqe-rg%3D%3D%22%5D%5D; __gads=ID=b5947f16edb0403d:T=1715364320:RT=1715366201:S=ALNI_MaJIyMoeqZaO--UX-FZHBggRDNdJA; __gpi=UID=00000a257e3bc349:T=1715364320:RT=1715366201:S=ALNI_MbCel5XYfhe-MDCNwKoU5oysKgFOA; __eoi=ID=934a22df78350c96:T=1715364320:RT=1715366201:S=AA-AfjZuN6AaSh_XAQiV6aYPqXtz; _ga_3KF4XTPHC4=GS1.1.1715364313.1.1.1715366296.60.0.0; _ga_QH2YGS7BB4=GS1.1.1715364313.1.1.1715366296.0.0.0; _ga_HNQ9P9MGZR=GS1.1.1715364319.1.1.1715366296.57.0.0',
    'dnt': '1',
    'if-none-match': 'W/"f9dc1f6f67"',
    'priority': 'u=1, i',
    'referer': 'https://www.sofascore.com/',
    'sec-ch-ua': '"Chromium";v="124", "Google Chrome";v="124", "Not-A.Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    'x-requested-with': '77b62c',
}

response = requests.get(
    'https://www.sofascore.com/api/v1/sport/football/scheduled-events/2024-05-11',
    cookies=cookies,
    headers=headers,
)

In [3]:
# Now we can get the data we want
today_games = response.json()


In [4]:
"""odds = requests.get('https://www.sofascore.com/api/v1/sport/football/odds/1/2024-05-10', cookies=cookies, headers=headers)

# Now we can get the data we want
today_odds = odds.json()
today_odds"""

"odds = requests.get('https://www.sofascore.com/api/v1/sport/football/odds/1/2024-05-10', cookies=cookies, headers=headers)\n\n# Now we can get the data we want\ntoday_odds = odds.json()\ntoday_odds"

In [8]:
# Function to extract information from each dictionary and return as a dictionary
def extract_info(d):
    tournament_name = d['tournament']['name']
    category_name = d['tournament']['category']['name']
    status_description = d['status']['description']
    home_team_short_name = d['homeTeam']['shortName']
    away_team_short_name = d['awayTeam']['shortName']
    start_timestamp = d['startTimestamp']

    return {
        'Tournament Name': tournament_name,
        'Category Name': category_name,
        'Status Description': status_description,
        'Home Team Short Name': home_team_short_name,
        'Away Team Short Name': away_team_short_name,
        'Start Timestamp': start_timestamp
    }

matches = list(today_games.values())
matches = matches[0]

# Apply the extraction function to each dictionary in the list
extracted_data_list = [extract_info(d) for d in matches]

# Create DataFrame from the list of extracted data
df = pd.DataFrame(extracted_data_list)
df['Start Timestamp'] = pd.to_datetime(df['Start Timestamp'], unit='s')

df.head(15)

,Tournament Name,Category Name,Status Description,Home Team Short Name,Away Team Short Name,Start Timestamp
0,Premier League,England,Not started,Fulham,Man City,2024-05-11 11:30:00
1,Premier League,England,Not started,Bournemouth,Brentford,2024-05-11 14:00:00
2,Premier League,England,Not started,Everton,Sheffield Utd,2024-05-11 14:00:00
3,Premier League,England,Not started,Newcastle,Brighton,2024-05-11 14:00:00
4,Premier League,England,Not started,Tottenham,Burnley,2024-05-11 14:00:00
5,Premier League,England,Not started,West Ham,Luton,2024-05-11 14:00:00
6,Premier League,England,Not started,Wolves,Crystal Palace,2024-05-11 14:00:00
7,Premier League,England,Not started,Forest,Chelsea,2024-05-11 16:30:00
8,LaLiga,Spain,Ended,Alavés,Girona,2024-05-10 19:00:00
9,LaLiga,Spain,Not started,Mallorca,Las Palmas,2024-05-11 12:00:00
